In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import numpy.random as nr
import math
import os

In [28]:
%matplotlib inline
data = pd.read_csv('../input/train.csv')
print(data.head(3))
data.info()

         date  date_block_num  shop_id  item_id  item_price  item_cnt_day  \
0  28.12.2014              23       25    12179  959.000000           NaN   
1  23.04.2014              15       25    14042  149.000000           1.0   
2  03.03.2014              14       12    11373  106.615385          13.0   

        ID  
0  2317582  
1  1576962  
2  1547382  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2925849 entries, 0 to 2925848
Data columns (total 7 columns):
date              object
date_block_num    int64
shop_id           int64
item_id           int64
item_price        float64
item_cnt_day      float64
ID                int64
dtypes: float64(2), int64(4), object(1)
memory usage: 156.3+ MB


In [29]:
# Check for negative item_cnt_day 
data[data['item_cnt_day']<0]['item_cnt_day'].value_counts()

-1.0     7221
-2.0       78
-3.0       13
-5.0        4
-4.0        3
-6.0        2
-22.0       1
-9.0        1
-16.0       1
Name: item_cnt_day, dtype: int64

In [30]:
data_filtered=data.loc[data['item_cnt_day']>0]
data_filtered.info()
data=data_filtered

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2918524 entries, 1 to 2925848
Data columns (total 7 columns):
date              object
date_block_num    int64
shop_id           int64
item_id           int64
item_price        float64
item_cnt_day      float64
ID                int64
dtypes: float64(2), int64(4), object(1)
memory usage: 178.1+ MB


In [31]:
data[data['item_cnt_day']<0]['item_cnt_day'].value_counts()

Series([], Name: item_cnt_day, dtype: int64)

In [32]:
def drop_duplicate(data, sub_set):
    print('Before drop shape:', data.shape)
    before = data.shape[0]
    data.drop_duplicates(sub_set, keep='first', inplace=True)
    data.reset_index(drop=True, inplace=True)
    print('After drop shape:', data.shape)
    after = data.shape[0]
    print('Total Duplicate:', before - after)

In [33]:
sales = data
test = pd.read_csv('../input/test.csv')
subset_train = ['date', 'date_block_num', 'shop_id', 'item_id', 'item_cnt_day']
subset_test = ['date', 'date_block_num', 'shop_id', 'item_id']
drop_duplicate(sales, sub_set=subset_train)
drop_duplicate(test, sub_set=subset_test)

# unreasonable_data(sales['item_cnt_day'])
# unreasonable_data(sales['item_price'])

median = sales[(sales.shop_id == 32) & (sales.item_id == 2973) & (sales.date_block_num == 4) & (
        sales.item_price > 0)].item_price.median()
sales.loc[sales.item_price < 0, 'item_price'] = median
# sales['item_cnt_day'] = sales['item_cnt_day'].clip(0, 1000)
sales['item_cnt_day'] = sales['item_cnt_day'].clip(0, 20)
sales['item_price'] = sales['item_price'].clip(0, 300000)

# Якутск Орджоникидзе, 56
sales.loc[sales.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
sales.loc[sales.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
sales.loc[sales.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11
# =======================
# for f in sales.columns: 
#     if sales[f].dtype=='object': 
#         lbl = preprocessing.LabelEncoder() 
#         lbl.fit(list(sales[f].values)) 
#         sales[f] = lbl.transform(list(sales[f].values))

# for f in test.columns: 
#     if test[f].dtype=='object': 
#         lbl = preprocessing.LabelEncoder() 
#         lbl.fit(list(test[f].values)) 
#         test[f] = lbl.transform(list(test[f].values))

Before drop shape: (2918524, 7)
After drop shape: (2918502, 7)
Total Duplicate: 22
Before drop shape: (10000, 6)
After drop shape: (10000, 6)
Total Duplicate: 0


In [34]:
sales.to_csv('processed_train.csv', sep=',',index=False)
test.to_csv('processed_test.csv', sep=',',index=False)

In [35]:
data = pd.read_csv('processed_train.csv')
data[data['item_cnt_day']<0]['item_cnt_day'].value_counts()

Series([], Name: item_cnt_day, dtype: int64)

In [36]:
data.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,ID
0,23.04.2014,15,25,14042,149.000000,1.0,1576962
1,03.03.2014,14,12,11373,106.615385,13.0,1547382
2,08.12.2013,11,47,12107,599.000000,1.0,1255724
3,23.01.2015,24,29,3341,2599.000000,1.0,2372492
4,14.12.2014,23,21,16541,399.000000,1.0,2308797


In [37]:
test = pd.read_csv('processed_test.csv')

In [38]:
test.head()

,date,date_block_num,shop_id,item_id,item_price,ID
0,06.02.2015,25,31,11208,699.0,2472142
1,21.11.2013,10,31,19525,149.0,1044690
2,03.01.2015,24,42,16169,299.0,2367559
3,18.07.2014,18,42,2737,199.0,1793501
4,13.04.2014,15,2,15229,1199.0,1585639
